In [2]:
import time
import datetime as dt
import pandas as pd
import pandas_ta as ta
import yfinance as yf
import matplotlib.pyplot as plt
import os
pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
tickersdf = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers =  [i.replace('.','-') for i in tickersdf.Symbol.to_list()]
# Remove offending symbols as neede
tickers.remove('WRK')

In [ ]:
def plot_signals(df):
    frame = df.copy()
    plt.figure(figsize=(20,5))
    plt.scatter(frame.loc[buy].index, frame.loc[buy]['open'], marker='^',c='g')
    plt.scatter(frame.loc[sell].index, frame.loc[sell]['open'], marker='*',c='r')
    plt.plot(frame['close'], alpha=0.7)

In [25]:
from pathlib import Path
import pandas_market_calendars as mcal
import datetime as dt
symbol = 'AAPL'
file = f"datasets/{symbol}.csv.gz"
#df = pd.read_csv(file, index_col='Date', parse_dates=True)
df = pd.read_csv(file, parse_dates=True)
df.symbol = symbol
#df['symbol'] = symbol

def df_normalize(symbol, df)
    df.reset_index(inplace=True)
    df.columns = df.columns.str.lower()
    df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df.symbol = symbol
    return df

def need_update(df):  
    symbol = df.symbol
    last_ts = df.iloc[-1].name
    nyse = mcal.get_calendar('NYSE')
    schedule = nyse.schedule(start_date=last_ts, end_date=last_ts)
    last_ts = schedule.loc[last_ts].market_close
    next_close_ts = last_ts + dt.timedelta(days=1)

    now_ts = pd.Timestamp.utcnow()
    schedule = nyse.schedule(start_date=now_ts, end_date=now_ts).iloc[0]

    if next_close_ts >= schedule.market_close:
        return False
    # TODO: How does this handle weekends and mid day
    # end is not included
    delta_df = yf.download(symbol, progress=False, start=next_close_ts.strftime('%Y-%m-%d'))
#     delta_df.columns = delta_df.columns.str.lower()
#     delta_df['date'] = delta_df.index
#     delta_df.index = delta_df.date
    delta_df = df_normalize(symbol, delta_df)
    
    df = df.append(delta_df)
    df.symbol = symbol
    return df

df = need_update(df)
df

,date,open,high,low,close,adj close,volume
2005-01-03,2005-01-03,1.16,1.16,1.12,1.13,0.97,691992000
2005-01-04,2005-01-04,1.14,1.17,1.12,1.14,0.98,1096810400
2005-01-05,2005-01-05,1.15,1.17,1.14,1.15,0.99,680433600
2005-01-06,2005-01-06,1.15,1.16,1.13,1.15,0.99,705555200
2005-01-07,2005-01-07,1.16,1.24,1.16,1.24,1.06,2227450400
...,...,...,...,...,...,...,...
2021-07-22,2021-07-22,145.94,148.20,145.81,146.80,146.80,77338200
2021-07-23,2021-07-23,147.55,148.72,146.92,148.56,148.56,71361600
2021-07-26 00:00:00,2021-07-26 00:00:00,148.27,149.83,147.70,148.99,148.99,72269700
2021-07-27 00:00:00,2021-07-27 00:00:00,149.12,149.21,145.55,146.77,146.77,103580300


In [53]:
#%run rsi_momo.py
symbol = 'XLK'
df = load_symbol(symbol)
params = {'rsi_entry': 25, 'rsi_exit': 35, 'sma_period': 150, 'stop_loss_pct': 10, 'max_trade_days': 12, 'id': 1322 }
df = rsi_momo_strategy(symbol, df, params)
